<h2><b>Scraping data about fnatic (e-sport organization) in CS:GO with Selenium</h2></b>

<h3>Libraries</h3>

In [423]:
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from IPython.display import Image
import cmd

<h3>Status code</h3>

In [424]:
url = "https://www.hltv.org/results?team=4991"
response = requests.get(url)
response.status_code

200

<h3>How the site looks like</h3>

In [425]:
Image(url="hltv_cookies.png",width=1500, height=700)

In [426]:
Image(url="hltv.png",width=1500, height=800)

<h3>Code for getting data about teams, score, event and link to the game</h3>

In [427]:
PATH = 'C:\webdrivers\chromedriver.exe'
driver = webdriver.Chrome(executable_path=PATH)
driver.get(url)
driver.implicitly_wait(10)
cookies = driver.find_element(By.XPATH,"//button[@id='CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll']")
cookies.click()
s = []
driver.implicitly_wait(5)
#results = driver.find_elements(By.XPATH,".//div[@class='result-con']")#14 ostatnia strona
for i in range(1,16):
    results = driver.find_elements(By.XPATH,".//div[@class='result-con']")
    for result in results:
        team1 = result.find_element(By.XPATH,".//a/div[@class='result']//table//tbody//tr//td[1]").text
        team2 = result.find_element(By.XPATH,".//a/div[@class='result']//table/tbody//tr//td[3]").text
        score = result.find_element(By.XPATH,".//a//div[@class='result']//table/tbody//tr//td[2]").text
        event = result.find_element(By.XPATH,".//a//div[@class='result']//table//tbody//tr//td[4]").text
        game_link = result.find_element(By.XPATH,".//a").get_attribute('href')
        s.append({"team1":team1,
                  "team2":team2,
                  "score":score,
                  "event":event,
                  "game_link":game_link})
    driver.implicitly_wait(5)
    if i != 15:
        next_page = driver.find_element(By.XPATH,"(//a[@class='pagination-next '])[1]")
        next_page.click()
    print(f'Getting page {i}. Waiting...')
    time.sleep(2)


C:\Users\piotr\AppData\Local\Temp\ipykernel_12744\1627560556.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=PATH)


Getting page 1. Waiting...
Getting page 2. Waiting...
Getting page 3. Waiting...
Getting page 4. Waiting...
Getting page 5. Waiting...
Getting page 6. Waiting...
Getting page 7. Waiting...
Getting page 8. Waiting...
Getting page 9. Waiting...
Getting page 10. Waiting...
Getting page 11. Waiting...
Getting page 12. Waiting...
Getting page 13. Waiting...
Getting page 14. Waiting...
Getting page 15. Waiting...


In [428]:
df = pd.DataFrame(s)

<h3>Getting the date of every game - this part is more difficult, because a lot of games were played at the same day and it had to be ascertained that each date occurs the right amount of time. The logic behind this is that each date occurence was multiplied by the amount of time a particular game occured in results_date = driver.find_elements(By.XPATH,".//div[@class='results-sublist']"). So in the example below, the date (formatted as strings inside a list) was multpilied by 3 (the example date Febraury 27th 2013 occured 3 times), because 3 games were played at this time which implies that len(date.find_elements(By.XPATH,".//div[@class='result-con']") in results_date = 3</h3>

In [429]:
Image(url="przyklad.png",width=1500, height=700)

<h3>Getting each element from each list with dates</h3>

In [430]:
PATH = 'C:\webdrivers\chromedriver.exe'
driver = webdriver.Chrome(executable_path=PATH)
driver.get(url)
driver.implicitly_wait(10)
cookies = driver.find_element(By.XPATH,"//button[@id='CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll']")
cookies.click()
results_date = driver.find_elements(By.XPATH,".//div[@class='results-sublist']")
for date in results_date:
    list_date = (date.find_element(By.XPATH,".//div[@class='standard-headline']").text.strip() + " ").split(' Results') * len(date.find_elements(By.XPATH,".//div[@class='result-con']"))
    for i in list_date:
        game_date  = f'{i}'
        print(game_date)

C:\Users\piotr\AppData\Local\Temp\ipykernel_12744\123345807.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=PATH)


Results for May 30th 2023 
Results for May 29th 2023 
Results for May 18th 2023 
Results for May 18th 2023 
Results for May 18th 2023 
Results for May 17th 2023 
Results for May 16th 2023 
Results for May 15th 2023 
Results for May 14th 2023 
Results for May 13th 2023 
Results for May 13th 2023 
Results for April 19th 2023 
Results for April 18th 2023 
Results for April 17th 2023 
Results for April 7th 2023 
Results for April 6th 2023 
Results for April 6th 2023 
Results for April 3rd 2023 
Results for April 2nd 2023 
Results for April 1st 2023 
Results for March 31st 2023 
Results for March 30th 2023 
Results for March 21st 2023 
Results for March 14th 2023 
Results for February 27th 2023 
Results for February 26th 2023 
Results for February 25th 2023 
Results for February 24th 2023 
Results for February 23rd 2023 
Results for February 22nd 2023 
Results for February 7th 2023 
Results for February 6th 2023 
Results for February 5th 2023 
Results for February 3rd 2023 
Results for Febr

<h3>Scraping dates</h3>

In [431]:
PATH = 'C:\webdrivers\chromedriver.exe'
driver = webdriver.Chrome(executable_path=PATH)
driver.get(url)
driver.implicitly_wait(10)
cookies = driver.find_element(By.XPATH,"//button[@id='CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll']")
cookies.click()
d = []
driver.implicitly_wait(5)
#results = driver.find_elements(By.XPATH,".//div[@class='result-con']")#14 ostatnia strona
for page in range(1,16):
    results_date = driver.find_elements(By.XPATH,".//div[@class='results-sublist']")
    for date in results_date:
        list_date = (date.find_element(By.XPATH,".//div[@class='standard-headline']").text.strip() + " ").split(' Results') * len(date.find_elements(By.XPATH,".//div[@class='result-con']"))
        for i in list_date:
            game_date  = f'{i}'
            d.append({"game_date":game_date})
    driver.implicitly_wait(5)
    if page != 15:
        next_page = driver.find_element(By.XPATH,"(//a[@class='pagination-next '])[1]")
        next_page.click()
    print(f'Getting page {page}. Waiting...')
    time.sleep(2)

C:\Users\piotr\AppData\Local\Temp\ipykernel_12744\3104152274.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=PATH)


Getting page 1. Waiting...
Getting page 2. Waiting...
Getting page 3. Waiting...
Getting page 4. Waiting...
Getting page 5. Waiting...
Getting page 6. Waiting...
Getting page 7. Waiting...
Getting page 8. Waiting...
Getting page 9. Waiting...
Getting page 10. Waiting...
Getting page 11. Waiting...
Getting page 12. Waiting...
Getting page 13. Waiting...
Getting page 14. Waiting...
Getting page 15. Waiting...


In [432]:
df2 = pd.DataFrame(d)

In [433]:
df

,team1,team2,score,event,game_link
0,fnatic,9z,1 - 2,IEM Dallas 2023,https://www.hltv.org/matches/2364475/fnatic-vs...
1,fnatic,MOUZ,9 - 16,IEM Dallas 2023,https://www.hltv.org/matches/2364392/fnatic-vs...
2,fnatic,Looking4Org,2 - 0,Gamers8 2023 Europe Open Qualifier 1,https://www.hltv.org/matches/2364348/looking4o...
3,fnatic,500,19 - 16,Gamers8 2023 Europe Open Qualifier 1,https://www.hltv.org/matches/2364346/fnatic-vs...
4,fnatic,GTZ,16 - 6,Gamers8 2023 Europe Open Qualifier 1,https://www.hltv.org/matches/2364342/fnatic-vs...
...,...,...,...,...,...
1457,fnatic,ALTERNATE aTTaX,16 - 14,RADEON Offensive by 99Damage,https://www.hltv.org/matches/2199933/fnatic-vs...
1458,fnatic,MOUZ,11 - 16,RADEON Offensive by 99Damage,https://www.hltv.org/matches/2197272/mouz-vs-f...
1459,fnatic,mousesports,14 - 16,DreamHack Winter 2012,https://www.hltv.org/matches/2192672/mousespor...
1460,fnatic,ESC,16 - 7,DreamHack Winter 2012,https://www.hltv.org/matches/2192670/fnatic-vs...


In [434]:
df2

,game_date
0,Results for May 30th 2023
1,Results for May 29th 2023
2,Results for May 18th 2023
3,Results for May 18th 2023
4,Results for May 18th 2023
...,...
1457,Results for December 10th 2012
1458,Results for December 4th 2012
1459,Results for November 22nd 2012
1460,Results for November 22nd 2012


In [435]:
df['date'] = df2['game_date']

<h3>Final data frame</h3>

In [436]:
df

,team1,team2,score,event,game_link,date
0,fnatic,9z,1 - 2,IEM Dallas 2023,https://www.hltv.org/matches/2364475/fnatic-vs...,Results for May 30th 2023
1,fnatic,MOUZ,9 - 16,IEM Dallas 2023,https://www.hltv.org/matches/2364392/fnatic-vs...,Results for May 29th 2023
2,fnatic,Looking4Org,2 - 0,Gamers8 2023 Europe Open Qualifier 1,https://www.hltv.org/matches/2364348/looking4o...,Results for May 18th 2023
3,fnatic,500,19 - 16,Gamers8 2023 Europe Open Qualifier 1,https://www.hltv.org/matches/2364346/fnatic-vs...,Results for May 18th 2023
4,fnatic,GTZ,16 - 6,Gamers8 2023 Europe Open Qualifier 1,https://www.hltv.org/matches/2364342/fnatic-vs...,Results for May 18th 2023
...,...,...,...,...,...,...
1457,fnatic,ALTERNATE aTTaX,16 - 14,RADEON Offensive by 99Damage,https://www.hltv.org/matches/2199933/fnatic-vs...,Results for December 10th 2012
1458,fnatic,MOUZ,11 - 16,RADEON Offensive by 99Damage,https://www.hltv.org/matches/2197272/mouz-vs-f...,Results for December 4th 2012
1459,fnatic,mousesports,14 - 16,DreamHack Winter 2012,https://www.hltv.org/matches/2192672/mousespor...,Results for November 22nd 2012
1460,fnatic,ESC,16 - 7,DreamHack Winter 2012,https://www.hltv.org/matches/2192670/fnatic-vs...,Results for November 22nd 2012
